In [ ]:
import os
import sys
import numpy as np

IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)

from imatools.common import vtktools as vtku
from imatools.common import ioutils as iou

def calc_cog(pts, el):
    return [np.mean(pts[ee], 0) for ee in el]


def norm2(a): return np.linalg.norm(a)
def norm_vec(a): return a/norm2(a)


def remove_from_dirlist(dir_list, name_to_rm):
    if name_to_rm in dir_list:
        dir_list.remove(name_to_rm)

    return dir_list

def perc_above(arr, th) : 
    return np.sum(arr>th) / len(arr)

def perc_below(arr, th) : 
    return np.sum(arr<th) / len(arr)

In [ ]:
import pandas as pd

# Locations of hard drive based on platform
dirdic = {'macOS': '/Volumes/sandisk',
          'Linux': '/media/jsl19/sandisk',
          'Windows': 'D:/'}

tex_dic = {'macOS': '/Users/jsolislemus/Documents/TEX',
           'Linux': '/home/jsl19/Documents/tex'}

base_dir = iou.fullfile(dirdic[iou.chooseplatform()], '01_atrialfibres/06_Reproducibility/05_UserProjects')
comp_dir = iou.fullfile(base_dir, '011_comparisons')

ld = remove_from_dirlist(os.listdir(iou.fullfile(comp_dir,'C0')), 'COMPARISONS')
ld = remove_from_dirlist(ld, 'MAPPING')

CX = ['C' + str(n) for n in np.linspace(1,50,num=50, dtype=int)]

print(ld)

# Comparisons
Array of `inter/intra`

Array of `auto/manual/mixed`

In [ ]:
df_comp = pd.read_csv(iou.fullfile(comp_dir, 'comparisons_path.csv'))
mode = df_comp['mode']
mode = mode[np.linspace(0,98,num=50, dtype=int)]

processing_raw = df_comp['processing'].tolist()
processing_raw = [processing_raw[ix] + processing_raw[ix+1] for ix in range(50)]

a='automatic'
m='manual'

processing = [p.replace(a+a, a) for p in processing_raw]
processing = [p.replace(m+m, m) for p in processing]
processing = [p.replace(a+m, 'mixed') for p in processing]
processing = [p.replace(m+a, 'mixed') for p in processing]

In [ ]:
import seaborn as sns

debug = False

CX = ['C' + str(n) for n in np.linspace(0,49,num=50, dtype=int)]
FVX = ['f_endo', 
    'f_epi']
FSX = ['gradlat',
    'lat',
    'ps']

files_dic = {
    'scalar' : FSX, 
    'vector' : FVX
}

var_dic = {
    'scalar' : 'diff_abs', 
    'vector' : 'abs_dot_product'
}

fibres_list = ['1', 'l']

field_list = ['scalar', 'vector']

field = 'scalar'

odic = dict()
for fx in range(len(files_dic[field])) : 
    mean_var = np.zeros(50, dtype=float)
    std_var  = np.zeros(50, dtype=float)
    prc75_var = np.zeros(50, dtype=float)
    corr_var = np.zeros(50, dtype=float)

    for fibre in fibres_list : 
        file_name = files_dic[field][fx] + '_' + fibre 

        for cx in range(len(CX)) :  
            test_path = iou.fullfile(comp_dir, CX[cx], 'COMPARISONS', file_name + '.csv') 
            df = pd.read_csv(test_path, index_col=False)

            mean_var[cx] = np.mean(df[var_dic[field]])
            std_var[cx]  = np.std(df[var_dic[field]]) 
            prc75_var[cx]  = np.percentile(df[var_dic[field]], 75) 
            if files_dic[field][fx] == 'ps' : 
                CORR_MAT = np.corrcoef(df['ps_0'], df['ps_1'])
                corr_var[cx]  =  CORR_MAT[0,1]

        odic[files_dic[field][fx] + '_mean'] = mean_var
        odic[files_dic[field][fx] + '_std'] = std_var
        odic[files_dic[field][fx] + '_prc75'] = prc75_var
        if files_dic[field][fx] == 'ps':
            odic[files_dic[field][fx] + '_corr2'] = corr_var

    
odic['mode'] = mode
odic['processing'] = processing

o_df_inter_intra = pd.DataFrame(odic)

In [ ]:
import itertools 

mode_bis = ['atlas'] * 50 

for fx in range(len(files_dic[field])):
    mean_var = np.zeros(50, dtype=float)
    std_var = np.zeros(50, dtype=float)
    prc75_var = np.zeros(50, dtype=float)
    corr_var = np.zeros(50, dtype=float)

    ff = files_dic[field][fx]

    fileA = ff + '_1'
    fileB = ff + '_l'
    for cx in range(len(CX)) :
        pathA = iou.fullfile(comp_dir, CX[cx], 'COMPARISONS', fileA + '.csv')
        pathB = iou.fullfile(comp_dir, CX[cx], 'COMPARISONS', fileB + '.csv')
        dfA = pd.read_csv(pathA, index_col=False)
        dfB = pd.read_csv(pathB, index_col=False)

        measA = np.asarray(list(itertools.chain(dfA[ff+'_0'], dfA[ff+'_1'])))
        measB = np.asarray(list(itertools.chain(dfB[ff+'_0'], dfB[ff+'_1'])))

        compy = np.abs(measA-measB)

        mean_var[cx] = np.mean(compy)
        std_var[cx] = np.std(compy)
        prc75_var[cx] = np.percentile(compy, 75)
        
    odic[ff + '_mean'] = mean_var
    odic[ff + '_std'] = std_var
    odic[ff + '_prc75'] = prc75_var

odic['mode'] = mode_bis
odic['processing'] = processing

o_df_atlas = pd.DataFrame(odic)

In [ ]:

o_df = pd.concat([o_df_inter_intra, o_df_atlas])
o_df.to_csv(iou.fullfile(comp_dir, 'measurements.csv'))

In [ ]:
import seaborn as sns
sns.set_palette('Pastel1')
sns.boxplot(data=o_df, x='processing', y='gradlat_prc75', hue='mode')